In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.transforms import v2
from torch.optim.lr_scheduler import StepLR
from timm import create_model
import pandas as pd
import copy
import time
from tqdm import tqdm
import os

In [2]:
train_transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomResizedCrop(224, scale=(0.8, 1.0)),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ColorJitter(0.2, 0.2, 0.2, 0.1),
    v2.RandomRotation(15),
    v2.RandomErasing(p=0.3),
    v2.Normalize([0.5]*3, [0.5]*3)
])

val_transform = v2.Compose([
    v2.ToImage(),
    v2.Resize((224, 224)),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize([0.5]*3, [0.5]*3)
])

train_dataset = datasets.ImageFolder('C:/Users/liamcee/Documents/farbruh/dataset/rafdb/train', transform=train_transform)
test_dataset = datasets.ImageFolder('C:/Users/liamcee/Documents/farbruh/dataset/rafdb/test', transform=val_transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

num_classes = len(train_dataset.classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model = create_model('coatnet_0_rw_224.sw_in1k', pretrained=False, num_classes=num_classes)
model.load_state_dict(torch.load('FRcoatnet_weights(R)(A).pth'))
model = model.to(device)

C:\Users\liamcee\AppData\Local\Temp\ipykernel_26496\3769308168.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('FRcoatnet_weights(R)(A).

In [4]:
if os.path.exists("coatnet_FpRa_metrics(R)(A).csv"):
    df_metrics = pd.read_csv("coatnet_FpRa_metrics(R)(A).csv")
    start_epoch = len(df_metrics)
    
    train_losses = df_metrics['train_loss'].tolist()
    train_accuracies = df_metrics['train_accuracy'].tolist()
    test_losses = df_metrics['test_loss'].tolist()
    test_accuracies = df_metrics['test_accuracy'].tolist()
    
    best_acc = max(test_accuracies)
else:
    start_epoch = 0
    train_losses, train_accuracies = [], []
    test_losses, test_accuracies = [], []
    best_acc = 0.0

best_model_wts = copy.deepcopy(model.state_dict())

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = StepLR(optimizer, step_size=2, gamma=0.1)

In [6]:
train_losses = []
train_accuracies = []
test_losses = []
test_accuracies = []

best_acc = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

def train(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    loop = tqdm(loader, desc="Training")

    for images, labels in loop:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

        loop.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100.0 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    print(f"Train Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

def evaluate(model, loader, criterion, epoch):
    global best_acc, best_model_wts
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    loop = tqdm(loader, desc="Evaluating")

    with torch.no_grad():
        for images, labels in loop:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            loop.set_postfix(loss=loss.item(), acc=100.0 * correct / total)

    epoch_loss = running_loss / len(loader)
    epoch_acc = 100.0 * correct / total
    test_losses.append(epoch_loss)
    test_accuracies.append(epoch_acc)

    print(f"Test  Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

    if epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, 'best_coatnet_FR(A)final.pth')
        print(f"best model saved with accuracy: {best_acc:.2f}%")

In [7]:
epochs = 10
for epoch in range(start_epoch, start_epoch + epochs):
    print(f"\nEpoch {epoch+1}/{start_epoch + epochs}")
    start_time = time.time()

    train(model, train_loader, optimizer, criterion)
    evaluate(model, test_loader, criterion, epoch)

    scheduler.step()

    elapsed = time.time() - start_time
    print(f"Epoch Time: {elapsed:.2f} seconds")


Epoch 11/20


Training: 100%|██████████| 767/767 [02:33<00:00,  5.00it/s, acc=82.7, loss=0.525] 


Train Loss: 0.4894, Accuracy: 82.75%


Evaluating: 100%|██████████| 421/421 [00:34<00:00, 12.14it/s, acc=87.5, loss=0.022]   


Test  Loss: 0.3589, Accuracy: 87.50%
best model saved with accuracy: 87.50%
Epoch Time: 188.32 seconds

Epoch 12/20


...

Test  Loss: 0.2181, Accuracy: 93.42%
Epoch Time: 190.63 seconds

Epoch 20/20


Training: 100%|██████████| 767/767 [02:35<00:00,  4.95it/s, acc=94, loss=0.179]    


Train Loss: 0.1745, Accuracy: 94.02%


Evaluating: 100%|██████████| 421/421 [00:38<00:00, 11.06it/s, acc=93.3, loss=0.00223] 

Test  Loss: 0.2170, Accuracy: 93.34%
Epoch Time: 193.12 seconds


In [8]:
metrics = {
    'epoch': list(range(1, len(train_losses) + 1)),
    'train_loss': train_losses,
    'train_accuracy': train_accuracies,
    'test_loss': test_losses,
    'test_accuracy': test_accuracies
}

df_metrics = pd.DataFrame(metrics)
df_metrics.to_csv("coatnet_FpRa_metrics(R)(A)final.csv", index=False)

In [9]:
torch.save(model.state_dict(), 'FRcoatnet_weights(R)(A)final.pth')
torch.save(model, 'FRcoatnet_full(R)(A)final.pth')